- 머신러닝은 데이터 가공/변환, 모델 학습/예측, 그리고 평가의 프로세스로 구성
- 머신러닝 모델은 성능 평가 지표로 예측 성능을 평가할 수 있다.
- 머신러닝 모델 회귀인 경우 : 실제 값과 예측값의 오차 평균값에 기반
- 머신러닝 모델 분류인 경우 : 일반적으로 실제 결과 데이터와 예측 결과 데이터가 얼마나 정확하고 오류가 적게 발생하는가에 기반, 하지만 단순히 정확도만 가지고 판단했을 경우 잘못된 평가 결과에 빠질 수 있음, 분류의 경우 이진 분류 하는 경우가 많기 때문에 정확도 보다 다른 성능 평가 지표가 더 중요시 되는 경우가 많다.
    - 분류의 성능 평가 지표 : 정확도, 오차행렬, 정확도, 재현율, F1 스코어, ROC AUC

# 01. 정확도(Accuracy)
- 실제 데이터에서 예측 데이터가 얼마나 같은지를 판단하는 지표
- 정확도(accuracy) = 에측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
- 직관적으로 모델 예측 성능 나타내는 평가 지표

In [12]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    #fit()메서드는 아무것도 학습하지 않음.
    def fit(self, X, y=None):
        pass
    #predict()메서드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
    def predict(self, X):
        pred = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i]==1:
                pred[i]==0
            else:
                pred[i]==1
        return pred
    
from sklearn.preprocessing import LabelEncoder
import numpy as np

#Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

#머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

#레이블 인코딩 수행
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features=['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

#앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv('Data/titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived',axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

#위에서 생성한 Dummy Classifier를 예측해 학습/예측/평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train,y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는 : {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도는 : 0.6145


- 정확도는 불균형한 레이블 값 분포에서 ML 모델의 성능을 판단할 경우, 적합한 평가 지표가 아니다.

### MNIST 데이터 세트를 변환해 불균형한 데이터 세트로 만든 뒤에 정확도 지표 적용 시 어떠한 문제 발생하는지 확인

In [15]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    #입력값으로 들어오는 X데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X),1),dtype=bool)
    
#사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()

#digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 False이고 0으로 변환.
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=1)

In [16]:
#불균형한 레이블 데이터 분포도 확인.
print('레이블 테스트 세트 크기 : ', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

#Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는 : {:.3f}'.format(accuracy_score(y_test, fakepred)))

레이블 테스트 세트 크기 :  (450,)
테스트 세트 레이블 0과 1의 분포도
0    402
1     48
dtype: int64
모든 예측을 0으로 하여도 정확도는 : 0.893


- 잘못된 데이터를 사용하여도 높은 정확도가 나옴
- 정확도 평가 지표는 불균형한 레이블 데이터 세트에서는 성능 수치로 사용해서는 안된다.

# 02. 오차행렬(confusion matrix, 혼동행렬)
- 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고(confused) 있는지도 함께 보여주는 지표  
<img src="https://velog.velcdn.com/images/jjw9599/post/d230bf92-2a11-4fa8-b3fe-aadcce1fc443/image.png"></img>

- TN, FP, FN, TP는 예측 클래스와 실제 클래스의 Positive 결정값(값1)과 Negative 결정값(값0)의 결합에 따라 결정
- TN는 예측값을 Negative 값 0으로 예측했고 실제 값 역시 Negative 값 0
- FP는 예측값을 Positive 값 1로 예측했는데 실제 값은 Negative 값 0
- FN은 예측값을 Negative 값 0으로 예측했는데 실제 값은 Positive 값 1
- TP는 예측값을 Positive 값 1로 예측했는데 실제 값 역시 Positive 값 1

In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)

array([[402,   0],
       [ 48,   0]])

- 출력된 오차 행렬 : ndarray 형태
- TN 402, FP 0, FN 48, TP 0
- 정확도 = (TN+TP)/(TN+FP+FN+TP)
- 일반적으로 불균형한 레이블 클래스를 가지는 이진 분류 모델에서는 많은 데이터 중에서 중점적으로 찾아야 하는 매우 적은 수의 결과값에 Positive를 설정해 1값을 부여하고, 그렇지 않은 경우는 Negative로 0 값을 부여하는 경우가 많다. ex)사기O=positive, 사기X=negative, 암 양성=positive, 암 음성=negative

- 불균형한 이진 분류 데이터 세트에서는 Positive 데이터 건수가 매우 작기 때문에 데이터에 기반한 ML 알고리즘은 positive보다는 negative로 예측 정확도가 높아지는 경향이 발생한다.
- negative로 예측하는 경향이 더 강해져 TN은 매우 커지고 TP는 매우 작아지게 되고, negative로 예측할 때 정확도가 높기 때문에 FN은 매우 작고 Positive로 예측하는 경우가 작기 때문에 FP도 역시 매우 작아짐.
- 결과적으로 정확도 지표는 비대칭한 데이터 세트에서 positive에 대한 예측 정확도를 판단하지 못한 채 negative에 대한 예측 정확도만으로 분류의 정확도가 높게 나타나는 수치적인 판단 오류를 일으키게 된다.
- 정확도는 분류 모델의 성능을 측정할 수 있는 한가지 요소일 뿐이다.